In [1]:
import pandas as pd
import requests
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import re
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

In [2]:
#Import dataframes
rep_df =pd.read_csv('rep_merge_11202019.csv',thousands=',')
dem_df = pd.read_csv('dem_merge_11202019.csv',thousands=',')

In [3]:
#add a tag so we can tell these two apart
rep_df['tag']='rep'
dem_df['tag']='dem'

In [4]:
rep_df.head(3)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag
0,0,"\n$2,710 raised of $6,500",https://www.gofundme.com/steven-crowder-live-a...,"Dallas, TX",Steven Crowder Live at SMU!,"Created March 7, 2018",SMU College Republicans is bringing comedian S...,rep
1,1,"\n$12,857 raised of $10,000",https://www.gofundme.com/reopen-a-nc-republica...,"Brookline, MA",Dems help reopen a NC Repub office,"Created October 16, 2016",NOTE: We reached our goal and then some in les...,rep
2,2,"\n$13,171 raised of $20,000",https://www.gofundme.com/troopersanchez,"Mcallen, TX",Funds for Trooper Moises Sanchez,"Created April 7, 2019","￼Moises Sanchez, a Texas State Trooper, was sh...",rep


In [5]:
df=rep_df.append(dem_df)

In [6]:
# df=df.drop_duplicates(subset='link')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 999
Data columns (total 8 columns):
Unnamed: 0      2000 non-null int64
funds           2000 non-null object
link            2000 non-null object
location        2000 non-null object
title           1999 non-null object
created_date    1899 non-null object
summary         1828 non-null object
tag             2000 non-null object
dtypes: int64(1), object(7)
memory usage: 140.6+ KB


In [8]:
#it would be good to delete rows with different currencies
#splitting the funds raised into raised and goal
df=df[df['funds'].str.contains('\$')==True]
df['raised']=df.funds.str.extract(r"(.*) raised of")
df['goal']=df.funds.str.extract(r".* raised of (.*)")

In [9]:
df.head(3)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal
0,0,"\n$2,710 raised of $6,500",https://www.gofundme.com/steven-crowder-live-a...,"Dallas, TX",Steven Crowder Live at SMU!,"Created March 7, 2018",SMU College Republicans is bringing comedian S...,rep,"$2,710","$6,500"
1,1,"\n$12,857 raised of $10,000",https://www.gofundme.com/reopen-a-nc-republica...,"Brookline, MA",Dems help reopen a NC Repub office,"Created October 16, 2016",NOTE: We reached our goal and then some in les...,rep,"$12,857","$10,000"
2,2,"\n$13,171 raised of $20,000",https://www.gofundme.com/troopersanchez,"Mcallen, TX",Funds for Trooper Moises Sanchez,"Created April 7, 2019","￼Moises Sanchez, a Texas State Trooper, was sh...",rep,"$13,171","$20,000"


In [10]:
#I want to make a column with the state too
df['state']=df['location'].str.extract(r".*, ([A-Z]+)")
df['state']=df.state.str.replace(',','')


In [11]:
df.sample(5, random_state=0)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal,state
781,781,\n$0 raised of $1.0B,https://www.gofundme.com/end-student-debt-now,"Chelsea, MI",End Student Debt Now,"Created December 20, 2018",The simple explanation is the United States is...,rep,$0,$1.0B,MI
717,717,"\n$100 raised of $12,700",https://www.gofundme.com/bob-jones-for-mayor-o...,"Indianapolis, IN",Bob Jones for Mayor of City of Lawrence,"Created March 30, 2019",Hi I am Bob Jones and I’m running for Mayor of...,rep,$100,"$12,700",IN
739,739,\n$10 raised of $500,https://www.gofundme.com/american-workers-asso...,"Liberty, MO",American Workers Association nonprofit startup,"Created September 24, 2019",We are looking to start a nonprofit organizati...,rep,$10,$500,MO
218,218,\n$485 raised of $405,https://www.gofundme.com/get-bb-to-minnesota,"Sacramento, CA",Get B.B. to Minnesota,"Created June 8, 2019",Hi Folks ❤️I recently found out that I wasn’t ...,rep,$485,$405,CA
184,184,"\n$350 raised of $8,000",https://www.gofundme.com/help-me-stay-in-schoo...,"Farmington, MI",Help me return to school post-assault,"Created November 9, 2019",My name is Tatum Moleski. I’m a senior at Miam...,dem,$350,"$8,000",MI


# Trying to Troubleshoot Dates of My Scraper: A Tangent

In [12]:
#for some reason some dates are missing but I seem to be able to find the dates for the individual links
no_date_df=df[df['created_date'].isnull()==True]['link']

In [13]:
no_date_df

56     https://www.gofundme.com/wu-college-republican...
57                       https://www.gofundme.com/w5x8ws
70     https://www.gofundme.com/suburban-housewives-f...
75                   https://www.gofundme.com/1r9fd78sio
94     https://www.gofundme.com/inform-swing-state-re...
104    https://www.gofundme.com/support-our-senate-re...
112    https://www.gofundme.com/are-you-a-democrat-or...
117       https://www.gofundme.com/mp9ap-nursing-classes
129    https://www.gofundme.com/confuse-republicans-a...
155    https://www.gofundme.com/donate-if-you-are-a-t...
156                     https://www.gofundme.com/6p6x6i0
160                     https://www.gofundme.com/6ncniu0
161                     https://www.gofundme.com/6ncmjao
174                     https://www.gofundme.com/6mwr1ug
175                     https://www.gofundme.com/6lz3cg0
186    https://www.gofundme.com/hispanic-republicans-...
189    https://www.gofundme.com/urban-minority-republ...
192    https://www.gofundme.com

In [14]:
no_dates=[]

In [15]:
#trying to find the dates again
# for link in no_date_df:
#     dt= {}
#     response = requests.get(link)
#     doc = BeautifulSoup(response.text)
#     date=doc.find(class_='created-date')
#     print(link)
#     try:
#         print(date.text)
#         dt['link']=link
#         dt['created_date']=date.text
#         no_dates.append(dt)
#     except:
#         print(date)

In [16]:
no_dates_df=pd.DataFrame(no_dates)

In [17]:
no_dates_df.head(3)

""


In [18]:
# df=df.merge(no_dates_df,on='link',how='left')

KeyError: 'link'

In [20]:
df.head()

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal,state
0,0,"\n$2,710 raised of $6,500",https://www.gofundme.com/steven-crowder-live-a...,"Dallas, TX",Steven Crowder Live at SMU!,"Created March 7, 2018",SMU College Republicans is bringing comedian S...,rep,"$2,710","$6,500",TX
1,1,"\n$12,857 raised of $10,000",https://www.gofundme.com/reopen-a-nc-republica...,"Brookline, MA",Dems help reopen a NC Repub office,"Created October 16, 2016",NOTE: We reached our goal and then some in les...,rep,"$12,857","$10,000",MA
2,2,"\n$13,171 raised of $20,000",https://www.gofundme.com/troopersanchez,"Mcallen, TX",Funds for Trooper Moises Sanchez,"Created April 7, 2019","￼Moises Sanchez, a Texas State Trooper, was sh...",rep,"$13,171","$20,000",TX
3,3,"\n$2,551 raised of $3,000",https://www.gofundme.com/uncw-college-republic...,"Wilmington, NC",UNCW College Republicans CPAC 2020,"Created October 21, 2019",Hello! We are asking for donations to assist u...,rep,"$2,551","$3,000",NC
4,4,"\n$360 raised of $3,000",https://www.gofundme.com/college-republicans-a...,"Santa Cruz, CA",College Republicans at UCSC,"Created May 23, 2019",The College Republicans at UC Santa Cruz are p...,rep,$360,"$3,000",CA


In [19]:
# df['col_z'] = df['created_date_x'].where(df['created_date_x'].notnull(), df['created_date_y'])
# df = df.drop(['created_date_x','created_date_y'],axis=1)

KeyError: 'created_date_x'

In [21]:
df.tail(5)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal,state
995,995,"\n$0 raised of $5,000",https://www.gofundme.com/1mwdygbrao,"Burleson, TX",TRUMP 2020,"Created May 12, 2019",Let’s give these democrats a run for their mon...,dem,$0,"$5,000",TX
996,996,"\n$0 raised of $500,000",https://www.gofundme.com/Restore-Connecticut,"Hartford, CT",Unite Connecticut,"Created May 11, 2019",Hi my name is Thomas Gilmer.I am running for U...,dem,$0,"$500,000",CT
997,997,"\n$0 raised of $100,000",https://www.gofundme.com/ujsrhm-build-that-wall,"Fayetteville, NC",BUILD THAT WALL,NaN,NaN,dem,$0,"$100,000",NC
998,998,"\n$0 raised of $5,000",https://www.gofundme.com/cte-kelli-kalvinski,"Warren, MI",Committee to Elect Kelli R. Kalvinski,"Created May 9, 2019",It was suggested to me by a few people I trust...,dem,$0,"$5,000",MI
999,999,"\n$0 raised of $12,000",https://www.gofundme.com/righteous-leaders-wil...,"Lithonia, GA",The POTUS must take the Kingdom of Heaven!,"Created May 8, 2019",THE RIGHT-SIDE VIEW OF THE SPIRIT OF MAN LOOKI...,dem,$0,"$12,000",GA


# Post Troubleshooting

In [22]:
pd.set_option('display.max_colwidth', -1)

In [23]:
df[df.created_date.isnull()==True].link

56     https://www.gofundme.com/wu-college-republicans-giving-tuesday             
57     https://www.gofundme.com/w5x8ws                                            
70     https://www.gofundme.com/suburban-housewives-for-republican-party-2020     
75     https://www.gofundme.com/1r9fd78sio                                        
94     https://www.gofundme.com/inform-swing-state-republicans-about-border-crisis
104    https://www.gofundme.com/support-our-senate-republicans                    
112    https://www.gofundme.com/are-you-a-democrat-or-a-republican                
117    https://www.gofundme.com/mp9ap-nursing-classes                             
129    https://www.gofundme.com/confuse-republicans-and-democrats                 
155    https://www.gofundme.com/donate-if-you-are-a-true-republican               
156    https://www.gofundme.com/6p6x6i0                                           
160    https://www.gofundme.com/6ncniu0                                           
161 

In [24]:
#made a fix so that all links tested that have null created dates are just campaign not found
#will drop these rows because the campaigns are gone
df=df.dropna()

In [25]:
# df['created_date']=df.col_z.str.replace('Created','')
df['created_date']=df.created_date.str.replace('Created','')

In [26]:
df.head(3)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal,state
0,0,"\n$2,710 raised of $6,500",https://www.gofundme.com/steven-crowder-live-at-smu,"Dallas, TX",Steven Crowder Live at SMU!,"March 7, 2018","SMU College Republicans is bringing comedian Steven Crowder to SMU on Thursday, March 22 at 7pm in McFarlin Auditorium. Due to an extremely high demand to see Louder with Crowder live at SMU, we are renting the largest venue on SMU's campus. Unfortunately, the need to use McFarlin has significantly increased our expected cost of the event. Our organization needs help paying for the venue, security, and necessary equipment for the show in order to keep it at McFarlin and ensure we have the maximum possible attendance. Within four days of the Eventbrite being posted, the venue is already set to be at least half full. We are expecting to have at least 1200 attendees at the event. It would be extremely disappointing for them if we have to switch to a smaller venue, so we want to do everything we can to keep Louder With Crowder in the largest venue on campus. We are striving to provide the best experience for SMU students and the DFW community to have an entertaining, educational night by hearing Steven's conservative message. Any help we receive will be extremely appreciated by everyone planning to attend, as well as our organization. We hope that whether you donate or not, you come to the event and have a great time!",rep,"$2,710","$6,500",TX
1,1,"\n$12,857 raised of $10,000",https://www.gofundme.com/reopen-a-nc-republican-office-2ukuprzy,"Brookline, MA",Dems help reopen a NC Repub office,"October 16, 2016","NOTE: We reached our goal and then some in less than 40 minutes after we went public with this campaign. We stopped accepting donations then because our aim is to fund re-opening the bombed-out office, and we decided that we had gathered a reasonable amount for that purpose. There are many worth causes, partisan and non-partisan. For example, might we suggest contributing to a North Carolina classroom through DonorsChoose ? Here's the original campaign description,DESCRIPTIONOn October 16, a North Carolina headquarters of the Republican Party was firebombed , and an abhorrent threat was painted on its outside wallAs Democrats, we are starting this campaign to enable the Orange County, North Carolina Republican office to re-open as soon as possible. Until an investigation is undertaken, we cannot know who did this or why. No matter the result, this is not how Americans resolve their differences. We talk, we argue, sometimes we march, and most of all we vote. We do not resort to violence by individuals or by mobs.So, let's all pitch in , no matter what your party affiliation, in and get that office open again quickly.More InfoHere's some additional information that GoFundMe likes to see expressed:My name is David Weinberger. (There's more about me here and here.) My name is on this campaign, but it's actually the work of several people:Dan GillmorJeff JarvisClay ShirkyJoe TrippiZeynep TufekciAll of us are long-time Internet students and do-ers. We're also all either Democrats, or independents who are leaning that way in this cycle.We put this campaign together quickly after talking about an idea that Zeynep tweeted . The idea evolved to the point where a GFM campaign made sense. I initiated it. Our aim is, as stated, to help fund the re-opening of the bombed-out office. We do not want any of the funds gathered to be used for other political activities. We are currently talking with officials of Orange County and hope soon to be talking with the local GOP leaders to figure out how to provide this funding in a way that meets the election laws (which are complex) and that accomplishes the campaign's stated goal of re-opening the office. If there's no legal, practical way to put all of the money to that use, we will find a strictly non-partisan way of using it, and will let the donors know so that they can partic

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1753 entries, 0 to 999
Data columns (total 11 columns):
Unnamed: 0      1753 non-null int64
funds           1753 non-null object
link            1753 non-null object
location        1753 non-null object
title           1753 non-null object
created_date    1753 non-null object
summary         1753 non-null object
tag             1753 non-null object
raised          1753 non-null object
goal            1753 non-null object
state           1753 non-null object
dtypes: int64(1), object(10)
memory usage: 164.3+ KB


In [28]:
df.head(3)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal,state
0,0,"\n$2,710 raised of $6,500",https://www.gofundme.com/steven-crowder-live-at-smu,"Dallas, TX",Steven Crowder Live at SMU!,"March 7, 2018","SMU College Republicans is bringing comedian Steven Crowder to SMU on Thursday, March 22 at 7pm in McFarlin Auditorium. Due to an extremely high demand to see Louder with Crowder live at SMU, we are renting the largest venue on SMU's campus. Unfortunately, the need to use McFarlin has significantly increased our expected cost of the event. Our organization needs help paying for the venue, security, and necessary equipment for the show in order to keep it at McFarlin and ensure we have the maximum possible attendance. Within four days of the Eventbrite being posted, the venue is already set to be at least half full. We are expecting to have at least 1200 attendees at the event. It would be extremely disappointing for them if we have to switch to a smaller venue, so we want to do everything we can to keep Louder With Crowder in the largest venue on campus. We are striving to provide the best experience for SMU students and the DFW community to have an entertaining, educational night by hearing Steven's conservative message. Any help we receive will be extremely appreciated by everyone planning to attend, as well as our organization. We hope that whether you donate or not, you come to the event and have a great time!",rep,"$2,710","$6,500",TX
1,1,"\n$12,857 raised of $10,000",https://www.gofundme.com/reopen-a-nc-republican-office-2ukuprzy,"Brookline, MA",Dems help reopen a NC Repub office,"October 16, 2016","NOTE: We reached our goal and then some in less than 40 minutes after we went public with this campaign. We stopped accepting donations then because our aim is to fund re-opening the bombed-out office, and we decided that we had gathered a reasonable amount for that purpose. There are many worth causes, partisan and non-partisan. For example, might we suggest contributing to a North Carolina classroom through DonorsChoose ? Here's the original campaign description,DESCRIPTIONOn October 16, a North Carolina headquarters of the Republican Party was firebombed , and an abhorrent threat was painted on its outside wallAs Democrats, we are starting this campaign to enable the Orange County, North Carolina Republican office to re-open as soon as possible. Until an investigation is undertaken, we cannot know who did this or why. No matter the result, this is not how Americans resolve their differences. We talk, we argue, sometimes we march, and most of all we vote. We do not resort to violence by individuals or by mobs.So, let's all pitch in , no matter what your party affiliation, in and get that office open again quickly.More InfoHere's some additional information that GoFundMe likes to see expressed:My name is David Weinberger. (There's more about me here and here.) My name is on this campaign, but it's actually the work of several people:Dan GillmorJeff JarvisClay ShirkyJoe TrippiZeynep TufekciAll of us are long-time Internet students and do-ers. We're also all either Democrats, or independents who are leaning that way in this cycle.We put this campaign together quickly after talking about an idea that Zeynep tweeted . The idea evolved to the point where a GFM campaign made sense. I initiated it. Our aim is, as stated, to help fund the re-opening of the bombed-out office. We do not want any of the funds gathered to be used for other political activities. We are currently talking with officials of Orange County and hope soon to be talking with the local GOP leaders to figure out how to provide this funding in a way that meets the election laws (which are complex) and that accomplishes the campaign's stated goal of re-opening the office. If there's no legal, practical way to put all of the money to that use, we will find a strictly non-partisan way of using it, and will let the donors know so that they can partic

In [29]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return float(x)

In [30]:
df['raised']=df.raised.str.replace('$','')
df['raised']=df.raised.str.replace(',','')
df['goal']=df.goal.str.replace('$','')
df['goal']=df.goal.str.replace(',','')

In [31]:
df['goal_int'] = df.goal.apply(value_to_float)
df['raised_int'] = df.raised.apply(value_to_float)


In [32]:
df.head(3)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal,state,goal_int,raised_int
0,0,"\n$2,710 raised of $6,500",https://www.gofundme.com/steven-crowder-live-at-smu,"Dallas, TX",Steven Crowder Live at SMU!,"March 7, 2018","SMU College Republicans is bringing comedian Steven Crowder to SMU on Thursday, March 22 at 7pm in McFarlin Auditorium. Due to an extremely high demand to see Louder with Crowder live at SMU, we are renting the largest venue on SMU's campus. Unfortunately, the need to use McFarlin has significantly increased our expected cost of the event. Our organization needs help paying for the venue, security, and necessary equipment for the show in order to keep it at McFarlin and ensure we have the maximum possible attendance. Within four days of the Eventbrite being posted, the venue is already set to be at least half full. We are expecting to have at least 1200 attendees at the event. It would be extremely disappointing for them if we have to switch to a smaller venue, so we want to do everything we can to keep Louder With Crowder in the largest venue on campus. We are striving to provide the best experience for SMU students and the DFW community to have an entertaining, educational night by hearing Steven's conservative message. Any help we receive will be extremely appreciated by everyone planning to attend, as well as our organization. We hope that whether you donate or not, you come to the event and have a great time!",rep,2710,6500,TX,6500.0,2710.0
1,1,"\n$12,857 raised of $10,000",https://www.gofundme.com/reopen-a-nc-republican-office-2ukuprzy,"Brookline, MA",Dems help reopen a NC Repub office,"October 16, 2016","NOTE: We reached our goal and then some in less than 40 minutes after we went public with this campaign. We stopped accepting donations then because our aim is to fund re-opening the bombed-out office, and we decided that we had gathered a reasonable amount for that purpose. There are many worth causes, partisan and non-partisan. For example, might we suggest contributing to a North Carolina classroom through DonorsChoose ? Here's the original campaign description,DESCRIPTIONOn October 16, a North Carolina headquarters of the Republican Party was firebombed , and an abhorrent threat was painted on its outside wallAs Democrats, we are starting this campaign to enable the Orange County, North Carolina Republican office to re-open as soon as possible. Until an investigation is undertaken, we cannot know who did this or why. No matter the result, this is not how Americans resolve their differences. We talk, we argue, sometimes we march, and most of all we vote. We do not resort to violence by individuals or by mobs.So, let's all pitch in , no matter what your party affiliation, in and get that office open again quickly.More InfoHere's some additional information that GoFundMe likes to see expressed:My name is David Weinberger. (There's more about me here and here.) My name is on this campaign, but it's actually the work of several people:Dan GillmorJeff JarvisClay ShirkyJoe TrippiZeynep TufekciAll of us are long-time Internet students and do-ers. We're also all either Democrats, or independents who are leaning that way in this cycle.We put this campaign together quickly after talking about an idea that Zeynep tweeted . The idea evolved to the point where a GFM campaign made sense. I initiated it. Our aim is, as stated, to help fund the re-opening of the bombed-out office. We do not want any of the funds gathered to be used for other political activities. We are currently talking with officials of Orange County and hope soon to be talking with the local GOP leaders to figure out how to provide this funding in a way that meets the election laws (which are complex) and that accomplishes the campaign's stated goal of re-opening the office. If there's no legal, practical way to put all of the money to that use, we will find a strictly non-partisan way of using it, and will let the donors kn

In [33]:
#put in csv for later
df.to_csv('merged_cleaned_back_up_11202019.csv',index=False)

# Cutting All Campaigns That Are Not in the USA
Seems like we can 1) get rid of rows with different currencies other than $ and 2) get rid of rows that don't have state abbreviations

In [34]:
#list of USA States:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [35]:
df.state.unique()

array(['TX', 'MA', 'NC', 'CA', 'MO', 'PA', 'TN', 'IL', 'NY', 'MI', 'WA',
       'CT', 'FL', 'IN', 'SD', 'ON', 'AZ', 'MD', 'MN', 'WV', 'NM', 'HI',
       'OR', 'KS', 'OH', 'F', 'AL', 'NH', 'NV', 'ND', 'GA', 'VA', 'SC',
       'OK', 'NJ', 'AR', 'IA', 'LA', 'ME', 'NSW', 'CO', 'DC', 'KY', 'WY',
       'ID', 'QC', 'DE', 'UT', 'MT', 'PR', 'VT', 'WI', 'N', 'NE', 'SA',
       'JAL', 'MS', 'AUS', 'VI', 'U', 'AK', 'RI', 'MP', 'AU'],
      dtype=object)

In [36]:
nonstate=df[df.state.isin(states)==False]

In [37]:
nonstate.location.unique()

array(['Mississauga, ON', 'Coral Springs, Florida', 'Hunters Hill, NSW',
       'Québec, QC', 'San Juan, PR', 'Webster, New Hampshire',
       'Myrtle Bank, SA', 'Chapala Centro, JAL', 'Fort Erie, ON',
       'Drummoyne, NSW', 'Murwillumbah, NSW', 'Brisbane, AUS',
       'St Thomas, VI', 'Niagara Falls, ON',
       'Sittingbourne, South East England, United Kingdom', 'Toronto, ON',
       'Saipan, MP', 'Oshawa, ON', 'Frankston, VIC, AU'], dtype=object)

In [38]:
df.loc[df.location == 'Coral Springs, Florida', ['state']] = 'FL'

In [39]:
df.loc[df.location == 'Webster, New Hampshire', ['state']] = 'NH'

In [40]:
pd.options.display.max_colwidth = 100

In [41]:
print(df[df['location']=='Green Brook, US'].link)

Series([], Name: link, dtype: object)


In [42]:
df.loc[df.location == 'Green Brook, US', ['state']] = 'NJ'

In [43]:
df=df.loc[df.state.isin(states)]

In [44]:
df.head(3)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal,state,goal_int,raised_int
0,0,"\n$2,710 raised of $6,500",https://www.gofundme.com/steven-crowder-live-at-smu,"Dallas, TX",Steven Crowder Live at SMU!,"March 7, 2018","SMU College Republicans is bringing comedian Steven Crowder to SMU on Thursday, March 22 at 7pm ...",rep,2710,6500,TX,6500.0,2710.0
1,1,"\n$12,857 raised of $10,000",https://www.gofundme.com/reopen-a-nc-republican-office-2ukuprzy,"Brookline, MA",Dems help reopen a NC Repub office,"October 16, 2016",NOTE: We reached our goal and then some in less than 40 minutes after we went public with this c...,rep,12857,10000,MA,10000.0,12857.0
2,2,"\n$13,171 raised of $20,000",https://www.gofundme.com/troopersanchez,"Mcallen, TX",Funds for Trooper Moises Sanchez,"April 7, 2019","￼Moises Sanchez, a Texas State Trooper, was shot while responding to an accident. He is currentl...",rep,13171,20000,TX,20000.0,13171.0


In [45]:
df.to_csv('merged_final_11202019.csv',index=False)

# Looking at the Overlap
taking a look at which campaigns have the same links with a simple merge. we'll use some of the same cleaning


In [46]:
def drop_y(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)

In [47]:
intersect_df = pd.merge(dem_df,rep_df,on='link',how='inner',suffixes=('', '_y'))

In [48]:
#are there any fields that aren't the same?
# for index, row in intersect_df.iterrows():
#      # access data using column names
#     if row['title_x'] != row['title_y']:
#         print(row['title_x'],"isn't",row['title_y'] )
# #     print(index, row['funds_x'], row['funds_y'], row['title_x'])

In [49]:
intersect_df.head(10)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,Unnamed: 0_y,funds_y,location_y,title_y,created_date_y,summary_y,tag_y
0,6,"\n$810 raised of $75,000",https://www.gofundme.com/create-the-democrat-dossier,"Lakehead, CA",Create The Democrat Dossier,"Created June 17, 2018","Regardless of your political affiliation, you are probably interested in understanding the truth...",dem,288,"\n$810 raised of $75,000","Lakehead, CA",Create The Democrat Dossier,"Created June 17, 2018","Regardless of your political affiliation, you are probably interested in understanding the truth...",rep
1,26,\n$260 raised of $500,https://www.gofundme.com/cosby2019,"Adamsville, AL",Cosby For Exec. VP of AL Young Democrats,"Created April 22, 2019","I have asked for a lot over the years! With the S support of many of you, we have accomplished a...",dem,570,\n$260 raised of $500,"Adamsville, AL",Cosby For Exec. VP of AL Young Democrats,"Created April 22, 2019","I have asked for a lot over the years! With the S support of many of you, we have accomplished a...",rep
2,28,"\n$205 raised of $10,000",https://www.gofundme.com/help-progressive-democrats-win,"Lake George, MI",Help Progressive Democrats Win,"Created September 11, 2018",NaN,dem,586,"\n$205 raised of $10,000","Lake George, MI",Help Progressive Democrats Win,"Created September 11, 2018",NaN,rep
3,31,"\n$1,155 raised of $1,092",https://www.gofundme.com/democrats-abroad-voting-workshop-1019costa-rica,"Chapala Centro, JAL","Trip to Attend Voting Workshop, Costa Rica","Created September 23, 2019","The Lake Chapala chapter of Democrats Abroad, where I am vice chair, has voted the funds to send...",dem,594,"\n$1,155 raised of $1,092","Chapala Centro, JAL","Trip to Attend Voting Workshop, Costa Rica","Created September 23, 2019","The Lake Chapala chapter of Democrats Abroad, where I am vice chair, has voted the funds to send...",rep
4,43,"\n$50 raised of $1,000",https://www.gofundme.com/a-democrat-for-amash,"Omaha, NE",A Democrat for Amash,"Created May 28, 2019",Congressman Justin Amash stood up at a town hall tonight and defended his stance of impeachment ...,dem,721,"\n$50 raised of $1,000","Omaha, NE",A Democrat for Amash,"Created May 28, 2019",Congressman Justin Amash stood up at a town hall tonight and defended his stance of impeachment ...,rep
5,53,\n$0 raised of $1.0M,https://www.gofundme.com/voting-democrat,"Houston, TX",Voting Democrat????,Created 5 days ago,"\nHey everyone\n\nIf you plan on voting Democrat in 2020 , let me get some of that money before ...",dem,797,\n$0 raised of $1.0M,"Houston, TX",Voting Democrat????,Created 5 days ago,"\nHey everyone\n\nIf you plan on voting Democrat in 2020 , let me get some of that money before ...",rep
6,57,"\n$0 raised of $1,000",https://www.gofundme.com/republicans-versus-democrats,"Woodland Hills, CA",Republicans versus Democrats,"Created November 2, 2019",This is a social experiment to see who is more passionate about the United States Republicans ve...,dem,68,"\n$0 raised of $1,000","Woodland Hills, CA",Republicans versus Democrats,"Created November 2, 2019",This is a social experiment to see who is more passionate about the United States Republicans ve...,rep
7,58,"\n$0 raised of $100,000",https://www.gofundme.com/save-our-president-from-democrats-witch-hunt,"Mesquite, TX",Save Our President From Democrats Witch Hunt,"Created October 26, 2019","Before 45 got elected, the Democrats thought they had a smooth sailing and a presidency bequeath...",dem,825,"\n$0 raised of $100,000","Mesquite, TX",Save Our President From Democrats Witch Hunt,"Created October 26, 2019","Before 45 got elected, the Democrats thought they had a smooth sailing and a presidency bequeath...",rep
8,59,"\n$0 raised of $10,000",https://www.gofundme.com/help-democrats-buy-false-facebook-ads,"Vienna, VA",Help Democrats Buy False Facebook Ads,"Created October 23, 2019",\nRepublicans are buying ads on Facebook with lies and fake news. Facebook changed its ad polic...,dem,831,"\n$

In [50]:
drop_y(intersect_df)

In [51]:
intersect_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 0 to 260
Data columns (total 8 columns):
Unnamed: 0      261 non-null int64
funds           261 non-null object
link            261 non-null object
location        261 non-null object
title           261 non-null object
created_date    252 non-null object
summary         244 non-null object
tag             261 non-null object
dtypes: int64(1), object(7)
memory usage: 18.4+ KB


In [52]:
intersect_df=intersect_df[intersect_df['funds'].str.contains('\$')==True]
intersect_df['raised']=intersect_df.funds.str.extract(r"(.*) raised of")
intersect_df['goal']=intersect_df.funds.str.extract(r".* raised of (.*)")


In [53]:
intersect_df.head(5)

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal
0,6,"\n$810 raised of $75,000",https://www.gofundme.com/create-the-democrat-dossier,"Lakehead, CA",Create The Democrat Dossier,"Created June 17, 2018","Regardless of your political affiliation, you are probably interested in understanding the truth...",dem,$810,"$75,000"
1,26,\n$260 raised of $500,https://www.gofundme.com/cosby2019,"Adamsville, AL",Cosby For Exec. VP of AL Young Democrats,"Created April 22, 2019","I have asked for a lot over the years! With the S support of many of you, we have accomplished a...",dem,$260,$500
2,28,"\n$205 raised of $10,000",https://www.gofundme.com/help-progressive-democrats-win,"Lake George, MI",Help Progressive Democrats Win,"Created September 11, 2018",NaN,dem,$205,"$10,000"
3,31,"\n$1,155 raised of $1,092",https://www.gofundme.com/democrats-abroad-voting-workshop-1019costa-rica,"Chapala Centro, JAL","Trip to Attend Voting Workshop, Costa Rica","Created September 23, 2019","The Lake Chapala chapter of Democrats Abroad, where I am vice chair, has voted the funds to send...",dem,"$1,155","$1,092"
4,43,"\n$50 raised of $1,000",https://www.gofundme.com/a-democrat-for-amash,"Omaha, NE",A Democrat for Amash,"Created May 28, 2019",Congressman Justin Amash stood up at a town hall tonight and defended his stance of impeachment ...,dem,$50,"$1,000"


In [54]:
intersect_df['raised']=intersect_df.raised.str.replace('$','')
intersect_df['raised']=intersect_df.raised.str.replace(',','')
intersect_df['goal']=intersect_df.goal.str.replace('$','')
intersect_df['goal']=intersect_df.goal.str.replace(',','')

In [55]:
intersect_df.tail()

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal
256,966,"\n$0 raised of $10,000",https://www.gofundme.com/comedians-for-democracy,"Los Angeles, CA",Comedians for Democracy,"Created June 3, 2019","\nElection Day, 2016.\n\nDo you remember where you were when Donald Trump became President?\n\nI...",dem,0,10000
257,969,"\n$0 raised of $10,000",https://www.gofundme.com/floridians-for-film-and-tech-production,"Tampa, FL",Floridians For Film and Tech Production,"Created May 31, 2019","Just a decade ago, Florida was the #3 destination in the United States for film and television p...",dem,0,10000
258,971,\n$0 raised of $5.0M,https://www.gofundme.com/cash-reward-for-reading-mueller-report,"Charlottesville, VA",Mueller Report Moolah,"Created May 31, 2019","The Mueller Report ""deserves the attention of every American.""Republican, Democrat or Independen...",dem,0,5.0M
259,974,"\n$0 raised of $388,000",https://www.gofundme.com/ringg-leeder-conservative-comedy-show,"Irvine, CA",Ringg Leeder Conservative Comedy Show,"Created June 5, 2019",WHAT IS RINGG LEEDER?Ringg Leeder is unlike anything you’ve ever seen before. Ringg Leeder is a...,dem,0,388000
260,983,"\n$0 raised of $50,000",https://www.gofundme.com/RepublicansSavingAmerica,"Pittsburgh, PA",Black Republicans for President Trump,"Created May 24, 2019","\nMy name is William Perry, and I am a 35 year old African American. When I speak to a lot of ou...",dem,0,50000


In [56]:
intersect_df.dropna()

,Unnamed: 0,funds,link,location,title,created_date,summary,tag,raised,goal
0,6,"\n$810 raised of $75,000",https://www.gofundme.com/create-the-democrat-dossier,"Lakehead, CA",Create The Democrat Dossier,"Created June 17, 2018","Regardless of your political affiliation, you are probably interested in understanding the truth...",dem,810,75000
1,26,\n$260 raised of $500,https://www.gofundme.com/cosby2019,"Adamsville, AL",Cosby For Exec. VP of AL Young Democrats,"Created April 22, 2019","I have asked for a lot over the years! With the S support of many of you, we have accomplished a...",dem,260,500
3,31,"\n$1,155 raised of $1,092",https://www.gofundme.com/democrats-abroad-voting-workshop-1019costa-rica,"Chapala Centro, JAL","Trip to Attend Voting Workshop, Costa Rica","Created September 23, 2019","The Lake Chapala chapter of Democrats Abroad, where I am vice chair, has voted the funds to send...",dem,1155,1092
4,43,"\n$50 raised of $1,000",https://www.gofundme.com/a-democrat-for-amash,"Omaha, NE",A Democrat for Amash,"Created May 28, 2019",Congressman Justin Amash stood up at a town hall tonight and defended his stance of impeachment ...,dem,50,1000
5,53,\n$0 raised of $1.0M,https://www.gofundme.com/voting-democrat,"Houston, TX",Voting Democrat????,Created 5 days ago,"\nHey everyone\n\nIf you plan on voting Democrat in 2020 , let me get some of that money before ...",dem,0,1.0M
6,57,"\n$0 raised of $1,000",https://www.gofundme.com/republicans-versus-democrats,"Woodland Hills, CA",Republicans versus Democrats,"Created November 2, 2019",This is a social experiment to see who is more passionate about the United States Republicans ve...,dem,0,1000
7,58,"\n$0 raised of $100,000",https://www.gofundme.com/save-our-president-from-democrats-witch-hunt,"Mesquite, TX",Save Our President From Democrats Witch Hunt,"Created October 26, 2019","Before 45 got elected, the Democrats thought they had a smooth sailing and a presidency bequeath...",dem,0,100000
8,59,"\n$0 raised of $10,000",https://www.gofundme.com/help-democrats-buy-false-facebook-ads,"Vienna, VA",Help Democrats Buy False Facebook Ads,"Created October 23, 2019",\nRepublicans are buying ads on Facebook with lies and fake news. Facebook changed its ad polic...,dem,0,10000
9,69,"\n$0 raised of $500,000",https://www.gofundme.com/how-to-make-a-republican-a-democrat,"Lugoff, SC",How to make a Republican a Democrat,"Created August 8, 2019","So, here we go! I am conservative republican who has fallen on semi-hard times. Since, I have ha...",dem,0,500000
10,79,"\n$0 raised of $150,000",https://www.gofundme.com/nmn67u-republican-or-democrat,"Fort Payne, AL",family home and car,"Created June 11, 2019",not gonna give a sob story I just need money. want to buy a home for my family and maybe a nice ...,dem,0,150000
